In [12]:
"""
Vehicles Routing Problem (VRP) with Time Windows.

This notebook is adapted to show that putting constraints based on the solution of the unconstrained problem,
constraints that are respected in that original solution,
we can get unstable results.
We are also unstable with respect to the number of vehicles 
"""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import pandas as pd


def create_data_model(nb_vehicles=4,pickupdel_option=0, max_TW=25, test_TW=0):
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = [
        [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
        [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
        [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
        [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
        [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
        [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
        [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
        [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
        [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
        [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
        [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
        [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
        [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
        [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
        [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
        [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
        [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]

   
    max_ =max_TW
    data['time_windows'] = [
        (0, max_),  # depot
        (0, max_+ test_TW),  # 1
        (0, max_),  # 2
        (0, max_),  # 3
        (0, max_+ test_TW),  # 4
        (0, max_),  # 5
        (0, max_+ test_TW),  # 6
        (0, max_+ test_TW),  # 7
        (0, max_),  # 8
        (0, max_),  # 9
        (0, max_+ test_TW),  # 10
        (0, max_),  # 11
        (0, max_+ test_TW),  # 12
        (0, max_),  # 13
        (0, max_),  # 14
        (0, max_+ test_TW),  # 15
        (0, max_),  # 16
    ]
    
    data_pickups_deliveries_options = [[],
                                       [[8,2],[11,15],[4,1]],
                                       [[8,2],[11,15],[4,1], [13,12]],
                                       [[8,2],[11,15], [13,12]],
                                       [[16,14],[11,15]],
                                       [[16,14],[11,15],[4,1], [13,12]],
                                       [[16,14],[11,15],[4,1], [13,12], [8,2]],
                                       ]
#     print(len(data_pickups_deliveries_options))
    data['pickups_deliveries'] = data_pickups_deliveries_options[pickupdel_option]
    
    data['num_vehicles'] = nb_vehicles
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution,verbose, nb_vehicles, pickupdel_table, max_TW, test_TW ):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}: '.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
      
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        if verbose:
            print( plan_output)
        total_time += solution.Min(time_var)
    # print('\n' + 'Total time of all routes: {0}min - with {1}vehicles / pickups-deliveries: {2} '.format(total_time,nb_vehicles, str(pickupdel_table) ))
    return total_time, nb_vehicles, pickupdel_table, max_TW, test_TW

def main(verbose, pickupdel_option,nb_vehicles,max_TW, test_TW):
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model(nb_vehicles=nb_vehicles,pickupdel_option=pickupdel_option, max_TW=max_TW, test_TW=test_TW)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        30,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        routing.solver().Add(
            time_dimension.CumulVar(pickup_index) <=
            time_dimension.CumulVar(delivery_index))
    
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        all_results.append(print_solution(data, manager, routing, solution, verbose, nb_vehicles, pickupdel_table = data['pickups_deliveries'], max_TW=max_TW, test_TW = test_TW ))
    else:
        print('no solution')

all_results = []
df=pd.DataFrame()
if __name__ == '__main__':
    print('ORIGINAL SOLUTION - STRAIGHT FROM ORTOOL GUIDE SNIPPET')
    main(verbose= True, nb_vehicles=4, pickupdel_option=0, max_TW = 100, test_TW =0)
    for max_TW in [25]:
        for test_TW in [0,10]:
            for nb_vehicles in [2,3,4]:
                for pickupdel_option in range(7):
                    main(verbose= False, nb_vehicles=nb_vehicles, pickupdel_option=pickupdel_option, max_TW = max_TW, test_TW =test_TW)

                    df= pd.DataFrame(all_results)
    df.columns= ['Total Time', 'Nb veh.', 'Pickups/del', 'Standard TW max', 'add_maxTW']

# for nice display in jupyter notebook (we could as well do print(df))
# df

ORIGINAL SOLUTION - STRAIGHT FROM ORTOOL GUIDE SNIPPET
Route for vehicle 0: 0 Time(0,0) -> 0 Time(0,0)Time of the route: 0min

Route for vehicle 1: 0 Time(0,0) -> 0 Time(0,0)Time of the route: 0min

Route for vehicle 2: 0 Time(0,0) -> 13 Time(4,4) -> 12 Time(6,6) -> 11 Time(7,7) -> 15 Time(10,10) -> 3 Time(16,16) -> 4 Time(17,17) -> 1 Time(19,19) -> 7 Time(23,23) -> 0 Time(25,25)Time of the route: 25min

Route for vehicle 3: 0 Time(0,0) -> 5 Time(3,3) -> 8 Time(5,5) -> 6 Time(7,7) -> 2 Time(10,10) -> 10 Time(14,14) -> 16 Time(18,18) -> 14 Time(20,20) -> 9 Time(23,23) -> 0 Time(25,25)Time of the route: 25min



#### Table showing that the TOTAL TIME (objective function), is not stable, 
#### despite only introducing constraints that are respecting in the original solution. 
Particularly unstable once we introduce the time windows LOOSE constraints

In [10]:
df

,Total Time,Nb veh.,Pickups/del,Standard TW max,add_maxTW
0,50,4,[],100,0
1,50,2,[],25,0
2,50,2,"[[8, 2], [11, 15], [4, 1]]",25,0
3,50,2,"[[8, 2], [11, 15], [4, 1], [13, 12]]",25,0
4,50,2,"[[8, 2], [11, 15], [13, 12]]",25,0
5,50,2,"[[16, 14], [11, 15]]",25,0
6,50,2,"[[16, 14], [11, 15], [4, 1], [13, 12]]",25,0
7,50,2,"[[16, 14], [11, 15], [4, 1], [13, 12], [8, 2]]",25,0
8,50,3,[],25,0
9,50,3,"[[8, 2], [11, 15], [4, 1]]",25,0
